In [1]:
import pandas as pd
import importlib.util
import sklearn
import numpy as np
from pathlib import Path
import joblib
import os   

In [2]:
output_dir = "classification_results_2023/"
all_patient_data = pd.read_csv("Combined_AM_Data.csv")

In [3]:
all_patient_data.columns

Index(['id', 'date', 'age', 'sex', 'BMI', 'income', 'road_dist', 'cooking',
       'am_pef_org', 'y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in',
       'tempdiffin', 'humidiffin', 'pm25diffin', 'pm10', 'pm25', 'o3', 'no2',
       'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp', 'seap',
       'solrhr', 'solramnt', 'grdt', 'class'],
      dtype='object')

In [4]:
# Loading classification.py
spec = importlib.util.spec_from_file_location('classification', r"Classificationv3.py")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [5]:
predictColumn = "class"
minorityVariable = 0 
classes = all_patient_data[predictColumn].unique().tolist()

n_classes = len(classes)

if n_classes > 2 :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='saga', multi_class='multinomial')
else :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='auto')

cls = [ "DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        "K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        "Logit Regression"  , logit_reg_model,
        "Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];

In [6]:
EVALUATION_METRICS = ['Weighted Accuracy', 'Sensitivity/Recall', 'Precision_avg', 'F1_avg', 'Specificity', 'Precision', 'Precision_class1', 'F1', 'F1_class1']
REQUIRED_COLUMNS = ['income','road_dist','cooking','y_am_pef','tempin','humidin','pm25in','co2in','tempdiffin','humidiffin','pm25diffin','pm10','pm25','o3','no2','co','so2','temp','windsd','humid','varp','dewpt','airp','seap','solrhr','solramnt','grdt','class']

In [7]:
%run "Classificationv3.py"
%run "Balance.py"

In [8]:
def classification_func(file_name, predictCol, minorVar, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df):

    print('~~~~~~~~~~~~~~~~~', file_name,'~~~~~~~~~~~~~~~~~~~~~~')

    dataset_dir = Path(output_dir + patient_id+category_group +  "/diagrams/" + file_name + '/') 

    # Create Directory for this patient
    if not (dataset_dir.exists() and dataset_dir.is_dir()):
        os.makedirs(dataset_dir)

    save_path = output_dir+ patient_id+category_group +'/diagrams/' + file_name + '/sklearn_models_balanced_for_train_'

    CLASS_VAR = predictCol
    MINORITY_VAR = minorVar
    bestDT = module.Classify(patient_id_df, True, class_var=CLASS_VAR, minority_var=MINORITY_VAR, drawConfusionMatrix=True, classifiers=cls, scale=True, save_path=save_path, after_split=True)

    confusion_matrixes_file = save_path + "confusion_mat_" + category_group + "_"+ file_name
    module.save_confusion_matrix(bestDT[1], confusion_matrixes_file)
    roc_curve_file = save_path + "roc_curve_" + category_group + "_"+ file_name
    module.save_Draw_Roc(bestDT[2], roc_curve_file)
    if len(bestDT[3])!=0:
      featureImportance_path = save_path + "important_mat_" + category_group + "_"+ file_name
      module.save_Draw_FeatureImportanceMatrix(patient_id_df,bestDT[3],cls[0], featureImportance_path, True)

    model_path = save_path + "bestModel_" + category_group + "_"+ file_name + ".joblib"
    joblib.dump(bestDT[4], model_path)

    df_evaluation_results[file_name] = bestDT[0]
    df_evaluation_results.to_csv(output_dir+patient_id+category_group+"/sklearn_models_balanced_for_train_metrics_"+category_group + ".csv")


In [9]:
def runner_decision_tree():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Decision Tree
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '1DecisionTree'
        cls = ["DecisionTree", sklearn.tree.DecisionTreeClassifier(max_depth=4)]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_knn():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # KNN
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '2K-NN'
        cls = ["K-NN", sklearn.neighbors.KNeighborsClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_lr():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Logistic Regression
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'LogisticRegression'
        print(logit_reg_model)
        cls = ["Logit Regression", logit_reg_model]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_svm():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # SVM
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'SVM'
        cls = ["SVM", sklearn.svm.SVC(kernel="linear")]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_nb():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Naive Bayes
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'NaiveBayes'
        cls = ["Naive Bayes", sklearn.gaussian_process.GaussianProcessClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)


In [10]:
#runner_decision_tree()

In [11]:
#runner_knn()

In [12]:
#runner_lr()

In [13]:
runner_svm()

~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.008785895071923733  ~~~~~~~
~~~~~~~ Validation Loss: 0.017104916274547577  ~~~~~~~
2/2 [==============================] - 1s 7ms/step
~~~~~~~ value_counts: ~~~~~~~
1    92
0    66
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  158  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.011267297901213169  ~~~~~~~
~~~~~~~ Validation Loss: 0.011553260497748852  ~~~~~~~
2/2 [==============================] - 0s 7ms/step
~~~~~~~ value_counts: ~~~~~~~
1    84
0    82
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  166  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  126  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.012712059542536736  ~~~~~~~
~~~~~~~ Validation Loss: 0.015687856823205948  ~~~~~~~
2/2 [==============================] - 0s 5ms/step
~~~

c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


* 1-----------kfold--------------- 2 ------------


c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


* 2*
{'C': 11, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1.501, 'decision_function_shape': 'ovr', 'degree': 5, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++
++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
~~~~~~~~~~~~~~~~~ SB-003 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.011445063166320324  ~~~~~~~
~~~~~~~ Validation Loss: 0.02052130177617073  ~~~~~~~
2/2 [==============================] - 0s 6ms/step
~~~~~~~ value_counts: ~~~~~~~
1    87
0    66
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  153  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~

...

* 2*
{'C': 21, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.501, 'decision_function_shape': 'ovr', 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++
++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
~~~~~~~~~~~~~~~~~ SB-112 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.012255183421075344  ~~~~~~~
~~~~~~~ Validation Loss: 0.03364073112607002  ~~~~~~~
1/1 [==============================] - 0s 114ms/step
~~~~~~~ value_counts: ~~~~~~~
1    94
0    52
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  146  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~

c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


----SVM kfold---------- 0.8 SVC(C=1, coef0=1.501, degree=4, kernel='poly')
* 1-----------kfold--------------- 2 ------------


c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sbhuv\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


* 2*
{'C': 41, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1.001, 'decision_function_shape': 'ovr', 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
------run_cv runs successfully------------
++++++++++++save_confusion_matrix++++++++++++
++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))


In [14]:
# runner_nb()